# Setup
(No need to read)

In [1]:
%%capture
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [4]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [5]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [6]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

# Analyze GPT-2-Small
80M parameter model

## Loading and Running Models

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


## Test prompts

## Adjective Identification

In [10]:
example_prompt = "Bob is blue. Mary is red. Bob is"
example_answer = " blue"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' is', ' blue', '.', ' Mary', ' is', ' red', '.', ' Bob', ' is']
Tokenized answer: [' blue']


Performance on answer token:
Rank: 0        Logit: 14.46 Prob: 10.27% Token: | blue|

Top 0th token. Logit: 14.46 Prob: 10.27% Token: | blue|
Top 1th token. Logit: 14.25 Prob:  8.35% Token: | white|
Top 2th token. Logit: 14.18 Prob:  7.74% Token: | a|
Top 3th token. Logit: 13.76 Prob:  5.11% Token: | green|
Top 4th token. Logit: 13.49 Prob:  3.90% Token: | red|
Top 5th token. Logit: 13.36 Prob:  3.43% Token: | yellow|
Top 6th token. Logit: 13.36 Prob:  3.42% Token: | black|
Top 7th token. Logit: 13.19 Prob:  2.88% Token: | purple|
Top 8th token. Logit: 12.94 Prob:  2.25% Token: | pink|
Top 9th token. Logit: 12.68 Prob:  1.72% Token: | the|


Ranks of the answer tokens: [(' blue', 0)]

Test if it is checking for the actual correct adjective, or if it's doing something else such as outputting the most recent adjective, the least recent adjective, etc. In any case, it is moving adjectives. Or perhaps not moving the same ones, but "identifying" similar ones.

In [9]:
example_prompt = "Bob is blue. Mary is red. Mary is"
example_answer = " red"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' is', ' blue', '.', ' Mary', ' is', ' red', '.', ' Mary', ' is']
Tokenized answer: [' red']


Performance on answer token:
Rank: 5        Logit: 15.00 Prob:  5.37% Token: | red|

Top 0th token. Logit: 16.12 Prob: 16.50% Token: | white|
Top 1th token. Logit: 15.77 Prob: 11.69% Token: | blue|
Top 2th token. Logit: 15.75 Prob: 11.40% Token: | green|
Top 3th token. Logit: 15.45 Prob:  8.48% Token: | yellow|
Top 4th token. Logit: 15.07 Prob:  5.79% Token: | black|
Top 5th token. Logit: 15.00 Prob:  5.37% Token: | red|
Top 6th token. Logit: 14.96 Prob:  5.19% Token: | purple|
Top 7th token. Logit: 14.86 Prob:  4.70% Token: | pink|
Top 8th token. Logit: 14.47 Prob:  3.16% Token: | orange|
Top 9th token. Logit: 14.15 Prob:  2.30% Token: | a|


Ranks of the answer tokens: [(' red', 5)]

This shows that it doesn't always work. Still, it "identifies color". Try different colors

In [11]:
example_prompt = "Bob is yellow. Mary is green. Bob is"
example_answer = " yellow"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' is', ' yellow', '.', ' Mary', ' is', ' green', '.', ' Bob', ' is']
Tokenized answer: [' yellow']


Performance on answer token:
Rank: 2        Logit: 14.22 Prob:  6.86% Token: | yellow|

Top 0th token. Logit: 15.10 Prob: 16.41% Token: | blue|
Top 1th token. Logit: 14.34 Prob:  7.72% Token: | red|
Top 2th token. Logit: 14.22 Prob:  6.86% Token: | yellow|
Top 3th token. Logit: 13.88 Prob:  4.87% Token: | a|
Top 4th token. Logit: 13.79 Prob:  4.47% Token: | orange|
Top 5th token. Logit: 13.65 Prob:  3.87% Token: | white|
Top 6th token. Logit: 13.61 Prob:  3.70% Token: | black|
Top 7th token. Logit: 13.53 Prob:  3.45% Token: | green|
Top 8th token. Logit: 13.47 Prob:  3.24% Token: | purple|
Top 9th token. Logit: 13.14 Prob:  2.33% Token: | pink|


Ranks of the answer tokens: [(' yellow', 2)]

This also fails. We can say it's not adjective movers.

Try different adjectives

In [12]:
example_prompt = "Bob is tall. Mary is green. Bob is"
example_answer = " tall"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' is', ' tall', '.', ' Mary', ' is', ' green', '.', ' Bob', ' is']
Tokenized answer: [' tall']


Performance on answer token:
Rank: 1        Logit: 14.28 Prob:  5.68% Token: | tall|

Top 0th token. Logit: 15.01 Prob: 11.76% Token: | a|
Top 1th token. Logit: 14.28 Prob:  5.68% Token: | tall|
Top 2th token. Logit: 13.39 Prob:  2.33% Token: | short|
Top 3th token. Logit: 13.33 Prob:  2.19% Token: | the|
Top 4th token. Logit: 13.20 Prob:  1.92% Token: | blue|
Top 5th token. Logit: 12.90 Prob:  1.43% Token: | big|
Top 6th token. Logit: 12.90 Prob:  1.42% Token: | skinny|
Top 7th token. Logit: 12.86 Prob:  1.37% Token: | very|
Top 8th token. Logit: 12.69 Prob:  1.16% Token: | small|
Top 9th token. Logit: 12.69 Prob:  1.16% Token: | not|


Ranks of the answer tokens: [(' tall', 1)]

In [13]:
example_prompt = "Bob is tall. Mary is big. Bob is"
example_answer = " tall"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' is', ' tall', '.', ' Mary', ' is', ' big', '.', ' Bob', ' is']
Tokenized answer: [' tall']


Performance on answer token:
Rank: 1        Logit: 13.91 Prob:  5.01% Token: | tall|

Top 0th token. Logit: 14.93 Prob: 13.91% Token: | a|
Top 1th token. Logit: 13.91 Prob:  5.01% Token: | tall|
Top 2th token. Logit: 13.19 Prob:  2.43% Token: | the|
Top 3th token. Logit: 12.85 Prob:  1.74% Token: | very|
Top 4th token. Logit: 12.83 Prob:  1.71% Token: | smart|
Top 5th token. Logit: 12.80 Prob:  1.65% Token: | not|
Top 6th token. Logit: 12.69 Prob:  1.47% Token: | an|
Top 7th token. Logit: 12.63 Prob:  1.39% Token: | big|
Top 8th token. Logit: 12.61 Prob:  1.36% Token: | short|
Top 9th token. Logit: 12.53 Prob:  1.26% Token: | strong|


Ranks of the answer tokens: [(' tall', 1)]

In [15]:
example_prompt = "Bob is big. Mary is small. Bob is"
example_answer = " big"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' is', ' big', '.', ' Mary', ' is', ' small', '.', ' Bob', ' is']
Tokenized answer: [' big']


Performance on answer token:
Rank: 1        Logit: 13.21 Prob:  4.11% Token: | big|

Top 0th token. Logit: 13.91 Prob:  8.27% Token: | a|
Top 1th token. Logit: 13.21 Prob:  4.11% Token: | big|
Top 2th token. Logit: 12.61 Prob:  2.26% Token: | the|
Top 3th token. Logit: 12.58 Prob:  2.20% Token: | very|
Top 4th token. Logit: 12.44 Prob:  1.92% Token: | not|
Top 5th token. Logit: 12.36 Prob:  1.76% Token: | tall|
Top 6th token. Logit: 12.36 Prob:  1.75% Token: | smart|
Top 7th token. Logit: 12.13 Prob:  1.40% Token: | always|
Top 8th token. Logit: 12.07 Prob:  1.32% Token: | strong|
Top 9th token. Logit: 11.94 Prob:  1.16% Token: | good|


Ranks of the answer tokens: [(' big', 1)]

In [16]:
example_prompt = "Bob is smart. Mary is strong. Bob is"
example_answer = " smart"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' is', ' smart', '.', ' Mary', ' is', ' strong', '.', ' Bob', ' is']
Tokenized answer: [' smart']


Performance on answer token:
Rank: 1        Logit: 13.81 Prob:  4.30% Token: | smart|

Top 0th token. Logit: 14.81 Prob: 11.68% Token: | a|
Top 1th token. Logit: 13.81 Prob:  4.30% Token: | smart|
Top 2th token. Logit: 13.43 Prob:  2.93% Token: | the|
Top 3th token. Logit: 13.21 Prob:  2.37% Token: | strong|
Top 4th token. Logit: 13.16 Prob:  2.25% Token: | funny|
Top 5th token. Logit: 13.00 Prob:  1.91% Token: | good|
Top 6th token. Logit: 12.77 Prob:  1.51% Token: | an|
Top 7th token. Logit: 12.76 Prob:  1.50% Token: | always|
Top 8th token. Logit: 12.73 Prob:  1.46% Token: | not|
Top 9th token. Logit: 12.66 Prob:  1.36% Token: | tough|


Ranks of the answer tokens: [(' smart', 1)]

## "Not" identification

In [17]:
example_prompt = "Bob is not smart. Mary is smart. Bob is"
example_answer = " not"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' is', ' not', ' smart', '.', ' Mary', ' is', ' smart', '.', ' Bob', ' is']
Tokenized answer: [' not']


Performance on answer token:
Rank: 0        Logit: 15.23 Prob: 26.17% Token: | not|

Top 0th token. Logit: 15.23 Prob: 26.17% Token: | not|
Top 1th token. Logit: 14.30 Prob: 10.28% Token: | smart|
Top 2th token. Logit: 14.09 Prob:  8.32% Token: | a|
Top 3th token. Logit: 12.65 Prob:  1.97% Token: | the|
Top 4th token. Logit: 12.18 Prob:  1.23% Token: | an|
Top 5th token. Logit: 12.08 Prob:  1.12% Token: | stupid|
Top 6th token. Logit: 12.05 Prob:  1.08% Token: | just|
Top 7th token. Logit: 12.01 Prob:  1.04% Token: | good|
Top 8th token. Logit: 12.01 Prob:  1.04% Token: | very|
Top 9th token. Logit: 12.00 Prob:  1.04% Token: | also|


Ranks of the answer tokens: [(' not', 0)]

In [18]:
example_prompt = "Bob is smart. Mary is not smart. Bob is"
example_answer = " smart"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' is', ' smart', '.', ' Mary', ' is', ' not', ' smart', '.', ' Bob', ' is']
Tokenized answer: [' smart']


Performance on answer token:
Rank: 0        Logit: 14.76 Prob: 14.05% Token: | smart|

Top 0th token. Logit: 14.76 Prob: 14.05% Token: | smart|
Top 1th token. Logit: 14.39 Prob:  9.66% Token: | a|
Top 2th token. Logit: 14.05 Prob:  6.84% Token: | not|
Top 3th token. Logit: 12.84 Prob:  2.06% Token: | the|
Top 4th token. Logit: 12.71 Prob:  1.81% Token: | good|
Top 5th token. Logit: 12.41 Prob:  1.34% Token: | an|
Top 6th token. Logit: 12.36 Prob:  1.27% Token: | very|
Top 7th token. Logit: 12.22 Prob:  1.10% Token: | funny|
Top 8th token. Logit: 12.16 Prob:  1.04% Token: | always|
Top 9th token. Logit: 12.13 Prob:  1.01% Token: | also|


Ranks of the answer tokens: [(' smart', 0)]

RESULT: It successfully outputs “smart” instead of “not”. 

ANALYSIS: So here, it’s not just outputting “not” because “not” exists.

In [21]:
example_prompt = "Bob is not big. Mary is big. Bob is"
example_answer = " not"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' is', ' not', ' big', '.', ' Mary', ' is', ' big', '.', ' Bob', ' is']
Tokenized answer: [' not']


Performance on answer token:
Rank: 0        Logit: 14.83 Prob: 20.93% Token: | not|

Top 0th token. Logit: 14.83 Prob: 20.93% Token: | not|
Top 1th token. Logit: 14.03 Prob:  9.42% Token: | a|
Top 2th token. Logit: 14.00 Prob:  9.10% Token: | big|
Top 3th token. Logit: 12.68 Prob:  2.45% Token: | the|
Top 4th token. Logit: 12.31 Prob:  1.68% Token: | small|
Top 5th token. Logit: 12.30 Prob:  1.67% Token: | very|
Top 6th token. Logit: 12.10 Prob:  1.36% Token: | bigger|
Top 7th token. Logit: 12.06 Prob:  1.31% Token: | also|
Top 8th token. Logit: 11.76 Prob:  0.97% Token: | really|
Top 9th token. Logit: 11.75 Prob:  0.96% Token: | an|


Ranks of the answer tokens: [(' not', 0)]

In [19]:
example_prompt = "Bob is big. Mary is big. Bob is"
example_answer = " big"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' is', ' big', '.', ' Mary', ' is', ' not', ' big', '.', ' Bob', ' is']
Tokenized answer: [' big']


Performance on answer token:
Rank: 0        Logit: 14.83 Prob: 22.01% Token: | big|

Top 0th token. Logit: 14.83 Prob: 22.01% Token: | big|
Top 1th token. Logit: 13.80 Prob:  7.86% Token: | a|
Top 2th token. Logit: 13.48 Prob:  5.71% Token: | not|
Top 3th token. Logit: 12.36 Prob:  1.85% Token: | the|
Top 4th token. Logit: 12.32 Prob:  1.79% Token: | bigger|
Top 5th token. Logit: 12.28 Prob:  1.72% Token: | small|
Top 6th token. Logit: 12.12 Prob:  1.46% Token: | very|
Top 7th token. Logit: 12.11 Prob:  1.44% Token: | tall|
Top 8th token. Logit: 11.87 Prob:  1.14% Token: | really|
Top 9th token. Logit: 11.79 Prob:  1.05% Token: | huge|


Ranks of the answer tokens: [(' big', 0)]

In [20]:
example_prompt = "Bob is not big. Mary is not big. Bob is"
example_answer = " not"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' is', ' not', ' big', '.', ' Mary', ' is', ' not', ' big', '.', ' Bob', ' is']
Tokenized answer: [' not']


Performance on answer token:
Rank: 0        Logit: 15.61 Prob: 41.19% Token: | not|

Top 0th token. Logit: 15.61 Prob: 41.19% Token: | not|
Top 1th token. Logit: 13.79 Prob:  6.65% Token: | a|
Top 2th token. Logit: 13.56 Prob:  5.30% Token: | big|
Top 3th token. Logit: 12.38 Prob:  1.62% Token: | small|
Top 4th token. Logit: 12.30 Prob:  1.50% Token: | the|
Top 5th token. Logit: 12.24 Prob:  1.42% Token: | very|
Top 6th token. Logit: 12.05 Prob:  1.17% Token: | bigger|
Top 7th token. Logit: 11.96 Prob:  1.07% Token: | just|
Top 8th token. Logit: 11.57 Prob:  0.73% Token: | also|
Top 9th token. Logit: 11.51 Prob:  0.69% Token: | an|


Ranks of the answer tokens: [(' not', 0)]